In [2]:
%load_ext autoreload
%autoreload 2


import nltk
import pandas as pd
import string
from nltk.corpus import stopwords 
from nltk.tokenize import TweetTokenizer



In [3]:
pd.set_option('max_colwidth', 100)

In [4]:
df = pd.read_csv('../raw_data/dilma.csv')



/tmp/ipykernel_10738/1274174624.py:1: DtypeWarning: Columns (1,2,5,6,7,8,9,10,11,12,13,14,15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../raw_data/dilma.csv')


In [4]:
data = pd.read_csv('../raw_data/biroliro_total_tweets.csv')

In [6]:
data.head().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 19 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   url              5 non-null      object
 1   date             5 non-null      object
 2   content          5 non-null      object
 3   id               5 non-null      int64 
 4   reply_count      5 non-null      int64 
 5   retweet_count    5 non-null      int64 
 6   like_count       5 non-null      int64 
 7   quote_count      5 non-null      int64 
 8   lang             5 non-null      object
 9   username         5 non-null      object
 10  displayname      5 non-null      object
 11  description      3 non-null      object
 12  verified         5 non-null      bool  
 13  created          5 non-null      object
 14  followers_count  5 non-null      int64 
 15  friends_count    5 non-null      int64 
 16  location         5 non-null      object
 17  protected        5 non-null      bool  

In [6]:
#filtro = df[(df.lang == 'pt') & (df.url.str.startswith('https'))]
#df = filtro.reset_index(drop = True)

In [108]:
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import  make_column_transformer
import re 

class CleaningEncoder(BaseEstimator, TransformerMixin):
    '''
    Receives raw text data from the tweets and returns clean, ready to process data:
    turns all into lower case;
    removes punctuation;
    removes stopwords;
    removes numbers;
    removes users' handles
    
    '''
    def __init__(self):
        pass
                        
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        #assert isinstance(X, pd.DataFrame)
        X_ = X.copy()      
        
        #removin NaN values
        X_ = X_.fillna('nenhuma descrição')
        
        #removing links
        X_ = X_.apply(lambda x: re.sub(r"http\S+", "", x))
        X_ = X_.apply(lambda x: re.sub(r"www.\S+", "", x))
                
        #removing punctuation from each tweet
        new_punc = list(string.punctuation)
        del new_punc[2]
        del new_punc[-11]
        for punctuation in new_punc:
            X_ = X_.str.replace(punctuation, '')
            
        #removing numbers
        X_ = X_.str.replace('\d+', '')
        
        #tokenizing - removes handles, applies lowercase, keeps #, shortens letter repetitions to three
        #ex: kkkkk, kkkkkk, kkkkkkkkk = kkk
        tkn = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True)
        X_ = X_.apply(lambda x: tkn.tokenize(x))
        
        #removing stopwords
        stop_words = stopwords.words('portuguese')
        stop_words.remove('não')
        addicional = [
            'ta', 'q', 'nao', 'tah', 'tao', 'eh', 'vc', 'voce',
            'pq', 'quedê', 'mane', 'mto', 'mt', 'bj', 'bjs',
            'b', 'sao', 'axo', 'mano', 'ae', 'neh', 'aí',
            'kkk', 'porque', 'né', 'no']
        stop_words.extend(addicional)
        
        X_ = X_.apply(lambda x: [word for word in x if word not in (stop_words)])
        
        return X_

In [109]:
#.str.replace('\d+', '')
X = data.content.head(40)
#X.str.lower()

#dataframe['column_name']=dataframe['column_name'].apply(str)



In [110]:
clean = CleaningEncoder()
tudopronto = clean.fit_transform(X)

tudopronto


/tmp/ipykernel_991/2400588789.py:39: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  X_ = X_.str.replace(punctuation, '')
/tmp/ipykernel_991/2400588789.py:42: FutureWarning: The default value of regex will change from True to False in a future version.
  X_ = X_.str.replace('\d+', '')


0                                                                              [vai, acontecer, biroliro]
1                                [passei, min, metrô, cercada, macho, babaca, falando, biroliro, inferno]
2     [tô, adotando, postura, ciro, pessoa, vem, tentar, convencer, biroliro, não, facista, mando, “, ...
3                       [facil, pro, eleitor, indeciso, escolher, ciro, haddad, biroliro, segundo, turno]
4                                                 [rompendo, laços, familiares, causa, biroliro, #sextou]
5        [alguma, dúvida, discurso, vai, ser, pt, destruiu, tanto, biroliro, consegue, arrumar, aguardem]
6                                                                  [carreata, biroliro, grande, duartina]
7     [“, tô, tão, desesperado, desiludido, possibilidade, biroliro, ser, eleito, domingo, não, voto, ...
8     [percebam, mundo, doido, hj, menina, fb, não, contato, anos, não, conhecia, bem, colocou, perfil...
9                                             

In [12]:
class TimeEncoder(BaseEstimator, TransformerMixin):
    '''
    Transforms the date column from string to datetime object 
    '''
    
    def __init__(self):
        pass

    def fit(self, X, y=None):
        X_ = X.copy()

SyntaxError: unexpected EOF while parsing (23576470.py, line 1)

In [76]:
X = data.content[0]
tkn = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True)
tkn.tokenize(X)

['o', 'que', 'vai', 'acontecer', 'com', 'o', 'biroliro']

In [ ]:
class Cleaner():
    def __init__(self, X)
    self.X = X
    
    def preproc_pipe(self):
        text_cols = ['content', 'description']

In [69]:
data

,url,date,content,id,reply_count,retweet_count,like_count,quote_count,lang,username,displayname,description,verified,created,followers_count,friends_count,location,protected,profile_img
0,https://twitter.com/cverenka1/status/998578871590105089,2018-05-21 14:58:50,@hallskkk O que vai acontecer com o biroliro,998578871590105089,1,0,1,0,pt,cverenka1,Verenka,NaN,False,2012-01-18 17:53:39,609,646,Nao tem,False,https://pbs.twimg.com/profile_images/1464951757453336576/lxd2k0ZW_normal.jpg
1,https://twitter.com/temposdegloriag/status/1048359692353896448,2018-10-05 23:50:02,"passei 15 min no metrô cercada de macho babaca falando do biroliro, nossa que inferno",1048359692353896448,0,0,0,0,pt,temposdegloriag,clara/ninha 🎾,também não sei de nada :D,False,2012-10-29 23:48:16,7155,5701,"sonza, groove, romania",False,https://pbs.twimg.com/profile_images/1529235096007585797/gvnJV9rU_normal.jpg
2,https://twitter.com/brunaceotto/status/1048359036863823872,2018-10-05 23:47:26,"tô adotando a postura do ciro, a pessoa vem tentar me convencer que biroliro não é facista e eu ...",1048359036863823872,0,6,27,0,pt,brunaceotto,bruna ceotto,"“eu, nós” está na amazon!",False,2009-04-14 12:50:57,2790,548,ela/dela,False,https://pbs.twimg.com/profile_images/1524799819130224646/_vcgCG8-_normal.jpg
3,https://twitter.com/adrianoaaguiar/status/1048357054551924736,2018-10-05 23:39:33,@_elidanobrega Mais facil pro eleitor indeciso escolher Ciro do que entre Haddad e o Biroliro no...,1048357054551924736,1,0,0,0,pt,adrianoaaguiar,Adriano Aguiar,"Cerveja, forró, geologia, e um Brasil mais empreendedor!",False,2010-09-16 00:06:33,295,437,Natal - RN,False,https://pbs.twimg.com/profile_images/1245194518414209025/BwZfyQLm_normal.jpg
4,https://twitter.com/brunaonilda/status/1048350124286599169,2018-10-05 23:12:01,Rompendo laços familiares por causa do biroliro #sextou https://t.co/PHxJS3ZpCI,1048350124286599169,0,0,2,0,pt,brunaonilda,silenzio bruna,NaN,False,2018-03-05 04:55:39,84,227,Brasil,False,https://pbs.twimg.com/profile_images/1424620987895660545/W1QpoqWq_normal.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3530,https://twitter.com/_midnightrain/status/1045349818896470018,2018-09-27 16:29:53,Como que vocês ousam dizer que esse homem é igual ao Biroliro???? https://t.co/FGU8zxqepB,1045349818896470018,1,0,0,0,pt,_midnightrain,Aninha do SUS 🚩,não me segue eu tô perdida,False,2009-08-24 00:22:45,240,269,NaN,False,https://pbs.twimg.com/profile_images/1461115222626902021/etZcmDGJ_normal.jpg
3531,https://twitter.com/lovesomeoness/status/1045338343201026049,2018-09-27 15:44:17,o biroliro o bonoro o bozonaro o bololo o boniro,1045338343201026049,0,0,0,0,pt,lovesomeoness,viúva do cebolinha 🇪🇪,grêmio é puro sentimento | YNWA | antifascista | administração - UFRGS,False,2018-06-10 02:35:25,466,419,"Porto Alegre, Brasil",False,https://pbs.twimg.com/profile_images/1530580434010132480/DtBt-6HB_normal.jpg
3532,https://twitter.com/carolbtr/status/1045309661640044544,2018-09-27 13:50:18,"pronto, achei o melhor apelido pro salnorabo: biroliro",1045309661640044544,1,0,11,0,pt,carolbtr,Carolina,"Fofoqueira por profissão 📚 Autora de ""O pôr do sol dos astronautas"". Contos nas coletâneas ""O va...",False,2009-02-13 02:48:24,2589,929,NaN,False,https://pbs.twimg.com/profile_images/1502260082767187970/SNhjObOT_normal.jpg
3533,https://twitter.com/rbcbittencourt/status/1045262200007544832,2018-09-27 10:41:43,o pessoal do biroliro já chegou credo,1045262200007544832,0,0,1,0,pt,rbcbittencourt,bi panic,medUFF ⚕ · ex cp2 · ☭ · uninviting but not half as impossible as everyone assumes · BA/RJ insta...,False,2015-09-15 18:40:13,695,416,fundo do poço,False,https://pbs.twimg.com/profile_images/1511738001109434372/e3Jz9Hoo_normal.jpg


In [ ]:
def clean(text):
    text = [w for w in text if not w.startswith('@')]
    text = ''.join(w for w in text)
    return text